In [6]:
!pip install langgraph
!pip install langchain_community
!pip install langchain_openai
!pip install --upgrade langgraph langchain langchain-core langchain-community
!pip install --upgrade openai pydantic typing_extensions



  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.14.0-py3-none-any.whl (43 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2


In [49]:
import os
from typing import Any, Annotated, Literal
from langchain_core.messages import ToolMessage, AIMessage, HumanMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import AnyMessage, add_messages
# from flask import Flask, request, jsonify
# from flask_cors import CORS
import json
import logging

# Configure Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment setup

OPENAI_API_KEY = os.getenv("OPEN_AI_API_KEY")
LLM_MODEL = os.getenv("LLM_MODEL")
# LLM_MODEL = "gpt-3.5-turbo"
llm = ChatOpenAI(model=LLM_MODEL, api_key=OPENAI_API_KEY, reasoning_effort= "high")


# FLASK_PORT = int(os.getenv("FLASK_PORT", 5010))  # Default to 5001
# FLASK_DEBUG = os.getenv("FLASK_DEBUG", "False").lower() == "true"

# Database and tools
db = SQLDatabase.from_uri("sqlite:///coffee_co.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables")
get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")

# Utility functions
def create_tool_node_with_fallback(tools: list) -> RunnableWithFallbacks[Any, dict]:
    """
    Create a ToolNode with a fallback to handle errors and surface them to the agent.
    """
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )

def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            ) for tc in tool_calls
        ]
    }

@tool
def db_query_tool(query: str) -> str:
    """
    Execute a SQL query against the database and get back the result.
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    """
    result = db.run_no_throw(query)
    if not result:
        return "Error: Query failed. Please rewrite your query and try again."
    return result

# Query check
query_check_system = """You are a SQL expert with a strong attention to detail.
Double check the SQLite query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any mistakes, rewrite the query. If not, reproduce the original query.
You will call the appropriate tool to execute the query after this check."""
query_check_prompt = ChatPromptTemplate.from_messages(
    [("system", query_check_system), ("placeholder", "{messages}")]
)
query_check = query_check_prompt | llm.bind_tools([db_query_tool], tool_choice="required")

# State definition
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Workflow
workflow = StateGraph(State)

# Add a node for the first tool call
def first_tool_call(state: State) -> dict[str, list[AIMessage]]:
    logger.info("State===>", state)
    return {
        "messages": [
            AIMessage(
                content="",
                tool_calls=[
                    {
                        "name": "sql_db_list_tables",
                        "args": {},
                        "id": "tool_abcd123",
                    }
                ],
            )
        ]
    }

# Nodes
workflow.add_node("first_tool_call",first_tool_call )
workflow.add_node("list_tables_tool", create_tool_node_with_fallback([list_tables_tool]))
workflow.add_node("get_schema_tool", create_tool_node_with_fallback([get_schema_tool]))
workflow.add_node("model_get_schema", lambda state: {"messages": [llm.bind_tools([get_schema_tool]).invoke(state["messages"])]})

# Describe a tool to represent the end state
class SubmitFinalAnswer(BaseModel):
    """Submit the final answer to the user based on the query results."""

    final_answer: str = Field(..., description="The final answer to the user")


query_gen_system = """You are a SQL expert with a strong attention to detail.
Based on the user's question and the database schema, generate a syntactically correct SQLite query to answer the question.
You can also use python and pandas
Only answer what is asked.
If a previous query resulted in an error, rewrite it to fix the mistake.
NEVER make up data if information is insufficient; say you don’t have enough info.
DO NOT make DML statements (INSERT, UPDATE, DELETE, DROP etc.).
"""


query_gen_prompt = ChatPromptTemplate.from_messages(
    [("system", query_gen_system), ("placeholder", "{messages}")]
)
query_gen = query_gen_prompt | llm
workflow.add_node("query_gen", lambda state: {"messages": [query_gen.invoke(state)]})

# Corrected model_check_query
workflow.add_node("correct_query", lambda state: {"messages": [query_check.invoke(state)]})

workflow.add_node("execute_query", create_tool_node_with_fallback([db_query_tool]))

# New formulate_final_answer node
final_answer_system = """Based on the query result, formulate the final answer to the user's question and call SubmitFinalAnswer with the final answer."""
final_answer_prompt = ChatPromptTemplate.from_messages(
    [("system", final_answer_system), ("placeholder", "{messages}")]
)
final_answer_gen = final_answer_prompt | llm.bind_tools([SubmitFinalAnswer], tool_choice="required")
workflow.add_node("formulate_final_answer", lambda state: {"messages": [final_answer_gen.invoke(state)]})

# Edges
workflow.add_edge(START, "first_tool_call")
workflow.add_edge("first_tool_call", "list_tables_tool")
workflow.add_edge("list_tables_tool", "model_get_schema")
workflow.add_edge("model_get_schema", "get_schema_tool")
workflow.add_edge("get_schema_tool", "query_gen")
workflow.add_edge("query_gen", "correct_query")
workflow.add_edge("correct_query", "execute_query")

def route_after_execute(state: State) -> Literal["formulate_final_answer", "query_gen"]:
    last_message = state["messages"][-1]
    if isinstance(last_message, ToolMessage) and not last_message.content.startswith("Error:"):
        return "formulate_final_answer"
    return "query_gen"

workflow.add_conditional_edges(
    "execute_query",
    route_after_execute,
    {"formulate_final_answer": "formulate_final_answer", "query_gen": "query_gen"}
)
workflow.add_edge("formulate_final_answer", END)

# Compile
sql_agent = workflow.compile()

# === Example Python Prompt ===
# query = "What is the latest date in orders"
# state = {"messages": [HumanMessage(content=query)]}
# output = sql_agent.invoke(state)
# result = output["messages"][-1].tool_calls[0]['args']['final_answer']
# print("\nFinal Answer:\n", result)



In [51]:
from datetime import date
import sqlite3
import pandas as pd

conn = sqlite3.connect('/Users/brunodupont/Desktop/proj/agent01/LangChainTutorial/coffee_co.db')
purch = pd.read_sql_query("SELECT * FROM purchases", conn)
latest_purchase_date = pd.to_datetime(purch['purchase_date']).max().date()
latest_day = purch.loc[pd.to_datetime(purch['purchase_date']) == pd.to_datetime(latest_purchase_date)]

current_qtr = (latest_purchase_date.month - 1) // 3 + 1
current_year, current_month, current_week = latest_purchase_date.year, latest_purchase_date.month, latest_purchase_date.isocalendar().week
current_iso_week_day = latest_day['purchase_iso_weekday'].max()

week_completion = 'no'
last_complete_week = current_week - 1
if current_iso_week_day == 7:
    week_completion = 'yes'
    last_complete_week = current_week



print(purch.columns)
print(latest_purchase_date)

# today = date.today()
# year = today.year
# week = today.isocalendar().week
# month = today.month
# quarter = (today.month - 1) // 3 + 1

current_timeframe = f"Year: {current_year}, Week: {current_week}. Last complete week: {last_complete_week}, day of week: {current_iso_week_day}, Current Week Complete: {week_completion}, Month: {current_month}, Quarter: Q{current_qtr}"

print(current_timeframe)


Index(['customer_id', 'purchase_date', 'product_name', 'type', 'price',
       'sub_discount', 'purchase_year', 'purchase_month', 'purchase_iso_week',
       'purchase_quarter', 'purchase_iso_weekday', 'shipping_time',
       'order_id'],
      dtype='object')
2025-07-07
Year: 2025, Week: 28. Last complete week: 27, day of week: 1, Current Week Complete: no, Month: 7, Quarter: Q3


In [ ]:
rag_query = f""" current timeframe a this time is as follows: {current_timeframe}
Use purchases and use purchase_iso_week (in orders) for weeks
Only use full weeks, ignore week in progress
don't give examples, just the actual numbers asked for
"""


INFO:__main__:State===>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Final Answer:
 Revenue for week 26 (the week before the last complete week) is 153302.44, and revenue for week 27 (the last complete week) is 155875.17.


In [62]:
query = f"""can you get revenue for YTD and QTD.
Compare to previous year
{rag_query}
"""

state = {"messages": [HumanMessage(content=query)]}
output = sql_agent.invoke(state)
result = output["messages"][-1].tool_calls[0]['args']['final_answer']
print("\nFinal Answer:\n", result)

INFO:__main__:State===>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Final Answer:
 YTD Revenue:
 • 2025: 3,662,276.79
 • 2024: 2,026,395.35

QTD Revenue (Q3):
 • 2025: 133,400.22
 • 2024: 86,839.24


In [64]:
query = f"""what is the top selling product YTD
Compare to previous year
{rag_query}
"""

state = {"messages": [HumanMessage(content=query)]}
output = sql_agent.invoke(state)
result = output["messages"][-1].tool_calls[0]['args']['final_answer']
print("\nFinal Answer:\n", result)

INFO:__main__:State===>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Final Answer:
 Brew Bliss Beans - Current Year (2025): 29,590 sales, Previous Year (2024): 16,413 sales


In [ ]:
#performance vs. forecast / plan
# find it in the forecast file
query = f"""what is the performance YTD against forecast
{rag_query}
"""

state = {"messages": [HumanMessage(content=query)]}
output = sql_agent.invoke(state)
result = output["messages"][-1].tool_calls[0]['args']['final_answer']
print("\nFinal Answer:\n", result)



INFO:__main__:State===>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Final Answer:
 YTD Actual Revenue: 3,645,893.69, YTD Forecast Revenue: 2,521,306.44, Revenue Variance: 1,124,587.25, Performance Percentage: 144.60%


In [75]:
purch

,customer_id,purchase_date,product_name,type,price,sub_discount,purchase_year,purchase_month,purchase_iso_week,purchase_quarter,purchase_iso_weekday,shipping_time,order_id
0,sub1,2020-07-09 00:00:00,Perk & Pour Grounds,sub,21.99,0.1,2020,7,28,3,4,8,0
1,sub1,2020-07-23 00:00:00,Perk & Pour Grounds,sub,21.99,0.1,2020,7,30,3,4,3,1
2,sub2,2020-07-09 00:00:00,Caffeine Cascade,sub,19.99,0.1,2020,7,28,3,4,2,2
3,sub3,2020-07-09 00:00:00,Roast Ritual Reserve,sub,19.99,0.1,2020,7,28,3,4,10,3
4,sub4,2020-07-09 00:00:00,Brew Bliss Beans,sub,19.99,0.1,2020,7,28,3,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
575558,alc34886,2025-07-06 00:00:00,Mornin’ Mojo Blend,alc,19.99,0.0,2025,7,27,3,7,2,642469
575559,alc34887,2025-07-06 00:00:00,Sunrise Sips,alc,19.99,0.0,2025,7,27,3,7,2,642470
575560,alc34888,2025-07-06 00:00:00,Perk & Pour Grounds,alc,21.99,0.0,2025,7,27,3,7,2,642478
575561,alc34889,2025-07-06 00:00:00,Mornin’ Mojo Blend,alc,19.99,0.0,2025,7,27,3,7,15,642480


In [73]:
query = f"""HOw is star rating trending this quarter,
Compare to previous quarter and last year
{rag_query}
"""

state = {"messages": [HumanMessage(content=query)]}
output = sql_agent.invoke(state)
result = output["messages"][-1].tool_calls[0]['args']['final_answer']
print("\nFinal Answer:\n", result)

INFO:__main__:State===>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Final Answer:
 Current Quarter: 3.53 star rating
Previous Quarter: 3.69 star rating
Last Year: 3.61 star rating


In [ ]:
query = f"""can you get revenue for the last week and the one before.
{rag_query}
"""

state = {"messages": [HumanMessage(content=query)]}
output = sql_agent.invoke(state)
result = output["messages"][-1].tool_calls[0]['args']['final_answer']
print("\nFinal Answer:\n", result)

In [76]:
# Need revenue target YTD and FY
# Compute time is long: can precompute, can add more intermediate tables
purch.loc[purch['purchase_iso_week'] <= 27].groupby(['purchase_year']).agg(revenue=('price', sum))

/var/folders/67/s2k3sk3x0wl0c9ytwvbr1ys80000gn/T/ipykernel_75606/182471095.py:3: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  purch.loc[purch['purchase_iso_week'] <= 27].groupby(['purchase_year']).agg(revenue=('price', sum))


,revenue
purchase_year,
2021,41331.59
2022,275140.15
2023,890417.09
2024,2026395.35
2025,3662276.79


In [72]:
purch.loc[(purch['purchase_year'] == 2024) & (purch['purchase_iso_week']<=27)].groupby(['product_name']
                                                                                  ).agg(orders=('order_id',lambda x: x.nunique())).sort_values('orders',ascending=False)



,orders
product_name,
Brew Bliss Beans,16413
Caffeine Cascade,13236
Java Journey Roast,13103
Roast Ritual Reserve,11250
Mornin’ Mojo Blend,9658
Perk & Pour Grounds,6569
Sunrise Sips,5030
Velvet Roast Co.,4977
Daily Drip Delight,4878


In [54]:
purch

,customer_id,purchase_date,product_name,type,price,sub_discount,purchase_year,purchase_month,purchase_iso_week,purchase_quarter,purchase_iso_weekday,shipping_time,order_id
0,sub1,2020-07-09 00:00:00,Perk & Pour Grounds,sub,21.99,0.1,2020,7,28,3,4,8,0
1,sub1,2020-07-23 00:00:00,Perk & Pour Grounds,sub,21.99,0.1,2020,7,30,3,4,3,1
2,sub2,2020-07-09 00:00:00,Caffeine Cascade,sub,19.99,0.1,2020,7,28,3,4,2,2
3,sub3,2020-07-09 00:00:00,Roast Ritual Reserve,sub,19.99,0.1,2020,7,28,3,4,10,3
4,sub4,2020-07-09 00:00:00,Brew Bliss Beans,sub,19.99,0.1,2020,7,28,3,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
575558,alc34886,2025-07-06 00:00:00,Mornin’ Mojo Blend,alc,19.99,0.0,2025,7,27,3,7,2,642469
575559,alc34887,2025-07-06 00:00:00,Sunrise Sips,alc,19.99,0.0,2025,7,27,3,7,2,642470
575560,alc34888,2025-07-06 00:00:00,Perk & Pour Grounds,alc,21.99,0.0,2025,7,27,3,7,2,642478
575561,alc34889,2025-07-06 00:00:00,Mornin’ Mojo Blend,alc,19.99,0.0,2025,7,27,3,7,15,642480


In [7]:
query = "list tables in the database"
state = {"messages": [HumanMessage(content=query)]}
output = sql_agent.invoke(state)
result = output["messages"][-1].tool_calls[0]['args']['final_answer']
print("\nFinal Answer:\n", result)

INFO:__main__:State===>
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Final Answer:
 The database contains the following tables: all_customers, conversions, orders, purchases, weekly_forecast.
